In [ ]:
%load_ext autoreload

import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import scipy as sp
import scipy.io as spio

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors as mcolors


import skimage.morphology as morph

import homology

mpl.rcParams['mathtext.fontset'] = 'cm'
sns.set_context('poster', font_scale=1.25)
sns.set(color_codes=True, palette='deep')
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})

In [ ]:
mat = spio.loadmat("../sample_data/Creases17.mat")

fig, ax = plt.subplots(figsize=(64,64))
ax.imshow(mat['ridges'], cmap=plt.cm.gray, interpolation=None)
ax.axis('off')
plt.show()

In [ ]:
# data = mat['ridges'][2400:2500, 1901:2000]
data = mat['ridges']

# data = np.array([[0, 0, 0],
#                  [0, 1, 0],
#                  [0, 0, 0],
#                  [0, 0, 0]])

pixels = []
dims = []
heights = []

curr_height = 0

num_zeros = np.count_nonzero(data==0)

print("Height:", curr_height)
print("Zeros:", num_zeros)

pixels.extend(np.nonzero(data.flatten())[0])
dims.extend(np.full_like(pixels, 2))
heights.extend(np.full_like(pixels, curr_height))

print(len(pixels))

# fig, ax = plt.subplots(figsize=(6,6))
# ax.imshow(data, cmap=plt.cm.gray)
# # ax.axis('off')
# plt.show()


max_radius = 4

base = data
prev_dilated = data
radius = 1

while num_zeros > 0:

    selem = morph.disk(radius)
    
#     print(selem)

    curr_height += 1

    dilated = morph.dilation(base, selem)

    new_nonzeros = np.nonzero((dilated - prev_dilated).flatten())[0]

    num_zeros -= len(new_nonzeros)

    print("Height:", curr_height, "Zeros:", num_zeros, "Radius", radius)

#     fig, ax = plt.subplots(figsize=(6,6))
#     ax.imshow(dilated, cmap=plt.cm.gray)
# #     ax.axis('off')
#     plt.show()

    pixels.extend(new_nonzeros)
    dims.extend(np.full_like(new_nonzeros, 2))
    heights.extend(np.full_like(new_nonzeros, curr_height))

    prev_dilated = dilated
    
    if radius == max_radius:
        base = dilated
        radius = 1
    else:
        radius += 1

#     if curr_height >= 40:
#         break
            
pixels.append(data.shape[0]*data.shape[1])
dims.append(0)
heights.append(curr_height+1)

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))

norm = mcolors.Normalize(vmin=0, vmax=150)
cmap = mpl.cm.Blues_r

ax.imshow(np.reshape(np.array(heights)[np.argsort(pixels)][:data.shape[0]*data.shape[1]], data.shape), 
          cmap=cmap, norm=norm)
#     ax.axis('off')
plt.show()

In [ ]:
print("Creating Triangulation")
comp = homology.pixel_triangulation(data.shape[0], data.shape[1], True)

print("Creating Filtration")
(simp_filt, dims) = homology.construct_lower_star_filtration(comp, pixels)

print("Calculating Persistence")
(ipairs, hsort, persist, sim_to_pindex) = homology.compute_persistence_pairs(simp_filt, dims, heights, comp)

print("Persistence Pairs:", ipairs)
print("Infinite Persistence:", persist)

In [ ]:
birth = [[] for i in range(3)]
death = [[] for i in range(3)]
mult = [[] for i in range(3)]
for d in range(3):
    for (i, j) in list(ipairs[d].keys()):
        birth[d].append(hsort[i])
        death[d].append(hsort[j])
        mult[d].append(ipairs[d][(i, j)])

pbirth = [[] for i in range(3)]
for d in range(3):
    for i in (persist[d].keys()):
        pbirth[d].append(hsort[i])

# print(birth)
# print(death)

for d in range(3):
    
    fig = plt.figure(figsize=(8,8))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], death[d], c=mult[d], marker='.', cmap='Blues', edgecolors='k', linewidths=1.0, s=100)
    
    ax1.scatter(pbirth[d], pbirth[d], marker='s', color='r')

    ax1.plot(np.linspace(0, curr_height, 100), np.linspace(0, curr_height, 100), 'k--')
    
    ax1.set_title(r"$d={}$".format(d))

#     ax1.set_xlim(1e1, 1e-7)
#     ax1.set_ylim(1e1, 1e-7)

    
#     ax1.set_xscale('log')
#     ax1.set_yscale('log')
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"death")
        
    plt.tight_layout()

    plt.show()

In [ ]:
%autoreload

print(mat['ridges'].shape)
data = mat['ridges'][2000:2250, 2000:2250]

Nx = data.shape[0]
Ny = data.shape[1]

print(Nx, Ny)

boundaries = {i+1:[] for i in range(2)}

for i in range(Ny+1):
    for j in range(Nx):
        boundaries[1].append([(Nx+1)*i + j,(Nx+1)*i + j+1])
        
for i in range(Ny):
    for j in range(Nx+1):
        boundaries[1].append([(Nx+1)*i + j,(Nx+1)*(i+1) + j])
        

for i in range(Ny):
    for j in range(Nx):
        boundaries[2].append([Ny*i + j, Ny*(Nx+1) + (Ny+1)*i + j+1, Ny*(i+1) + j, Ny*(Nx+1) + (Ny+1)*i + j])
        

In [ ]:
simplices = []
dims = []
heights = []

curr_height = 0

num_zeros = np.count_nonzero(data==0)

print("Height:", curr_height)
print("Zeros:", num_zeros)

simplices.extend(np.nonzero(data.flatten())[0])
dims.extend(np.full_like(simplices, 2))
heights.extend(np.full_like(simplices, curr_height))

print(len(simplices))

fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(data, cmap=plt.cm.gray)
ax.axis('off')
plt.show()


selem = morph.disk(4)
print(selem)

while num_zeros > 0:
    
    curr_height += 1

    
    dilated = morph.dilation(data, selem)
        
    new_nonzeros = np.nonzero((dilated - data).flatten())[0]
                
    num_zeros -= len(new_nonzeros)
    
    print("Height:", curr_height)
    print("Zeros:", num_zeros)
    
    fig, ax = plt.subplots(figsize=(6,6))
    ax.imshow(dilated, cmap=plt.cm.gray)
    ax.axis('off')
    plt.show()
        
    simplices.extend(new_nonzeros)
    dims.extend(np.full_like(new_nonzeros, 2))
    heights.extend(np.full_like(new_nonzeros, curr_height))
        
    data = dilated
    
    if curr_height >= 32:
        break


In [ ]:
(ipairs, pheight, persist, sim_to_pindex) = homology.compute_persistence_pairs(simplices, dims, heights, boundaries)

print(ipairs)
print(persist)

In [ ]:
birth = [[] for i in range(3)]
death = [[] for i in range(3)]
mult = [[] for i in range(3)]
for d in range(3):
    for (i, j) in list(ipairs[d].keys()):
        birth[d].append(pheight[i])
        death[d].append(pheight[j])
        mult[d].append(ipairs[d][(i, j)])

pbirth = [[] for i in range(3)]
for d in range(3):
    for i in (persist[d].keys()):
        pbirth[d].append(pheight[i])

# print(birth)
# print(death)

for d in range(3):
    
    fig = plt.figure(figsize=(12,4))
    
    ax1 = fig.add_subplot(1,1,1)

    ax1.scatter(birth[d], death[d], c=mult[d], marker='o', cmap='Blues', edgecolors='k', linewidths=1.0, s=100)
    
    ax1.scatter(pbirth[d], pbirth[d], marker='s', color='r')

    ax1.plot(np.linspace(0, curr_height, 100), np.linspace(0, curr_height, 100), 'k--')
    
    ax1.set_title(r"$d={}$".format(d))

#     ax1.set_xlim(1e1, 1e-7)
#     ax1.set_ylim(1e1, 1e-7)

    
#     ax1.set_xscale('log')
#     ax1.set_yscale('log')
    
    ax1.set_xlabel(r"birth")
    ax1.set_ylabel(r"death")
        
    plt.tight_layout()

    plt.show()